In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
from time import time 

# Load Time

In [ ]:
%%time
t_load_start = time()

from astropy.io import fits 

image = fits.getdata('./bg_subtracted_abell2744_test_image.fits')
header = fits.getheader('./bg_subtracted_abell2744_test_image.fits')
rms =  fits.getdata('./bg_subtracted_abell2744_test_rms.fits')

# Load PSF image (2D array)
PSF = fits.getdata('f105w_psf.fits.gz')

# Normalize PSF 
PSF = PSF / PSF.sum()

t_load_end = time()


In [ ]:
load_time = t_load_end - t_load_start # Seconds
load_time

In [ ]:
# Note that the PSF shape is odd on all sides
print("PSF Shape = {}".format(PSF.shape))

# Plot PSF and use vmax and vmin to show difraction spikes
plt.imshow(PSF, vmin=0, vmax=PSF.std()/10)
plt.show()

In [ ]:
import numpy as np


vmax = 0.05 # Use the image std as max and min of all plots 
vmin = - vmax 

plt.imshow(image, vmin=vmin, vmax=vmax)
plt.title("Galaxy in Abell 2744")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

## Make Source Catalog 

We will use the sigma clipped std as a threshold at the segmentation and deblending steps.


In [ ]:
from astropy.stats import sigma_clipped_stats
image_mean, image_median, image_stddev = sigma_clipped_stats(image.data, sigma=3)

Here we identity sources in the input image.

In [ ]:
from petrofit.segmentation import make_catalog, plot_segments

threshold = image_stddev * 2

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 2**2


cat, segm, segm_deblend = make_catalog(    
    image, 
    threshold=threshold, 
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(cat))

In [ ]:
from petrofit.photometry import order_cat
from petrofit.utils import plot_target
# Sort and select object of interest in the catalog
sorted_idx_list = order_cat(cat, key='area', reverse=True)
idx = sorted_idx_list[2] # index 0 is largest 
source = cat[idx]  # get source from the catalog 

plot_target(
    position=(source.maxval_xindex, source.maxval_yindex), 
    image=image, 
    size=max(image.data.shape)//2, 
    vmax=vmax, 
    vmin=vmin
)

In [ ]:
source.label

# StatMorph

In [ ]:
header['BUNIT']

In [ ]:
header['EXPTIME']

# Run Time

In [ ]:
run_time_start = time()
import statmorph

gain = header['EXPTIME']

source_morphs = statmorph.SourceMorphology(
    image, segm_deblend, label=5, gain=gain, psf=PSF, weightmap=rms)

run_time_end = time()

In [ ]:
run_time = run_time_end - run_time_start
run_time

In [ ]:
total_time = load_time + run_time 
total_time

In [ ]:
morph = source_morphs

In [ ]:
morph.label

In [ ]:
from astropy.table import Table
rows = ['amplitude', 'r_eff', 'n', 'x_0', 'y_0', 'ellip', 'theta', 'r_petro', 'r_20', 'r_80', 'C2080', 'Total Flux']

sm_results = [morph.sersic_amplitude,  morph.sersic_rhalf,  morph.sersic_n,  morph.sersic_xc, morph.sersic_yc,  morph.sersic_ellip, morph.sersic_theta, morph.rpetro_ellip, morph.r20, morph.r80, morph.concentration, morph.flux_ellip,]

sm_results = [float('{:0.3f}'.format(i)) for i in sm_results]

t = Table(data=[rows, sm_results], names=['Values', 'statmorph'])
t

In [ ]:
t.write('statmorph_abell2744.csv', overwrite=True)

In [ ]:
from petrofit.photometry import flux_to_abmag
flux_to_abmag(float(morph.flux_ellip), header)

In [ ]:
# snp = 100.0
# from astropy.visualization import simple_norm
# ny, nx = image.data.shape
# y, x = np.mgrid[0:ny, 0:nx]
# fitted_model = statmorph.ConvolvedSersic2D(
#     amplitude=morph.sersic_amplitude,
#     r_eff=morph.sersic_rhalf,
#     n=morph.sersic_n,
#     x_0=morph.sersic_xc,
#     y_0=morph.sersic_yc,
#     ellip=morph.sersic_ellip,
#     theta=morph.sersic_theta)
# fitted_model.set_psf(PSF)  # required when using ConvolvedSersic2D
# image_model = fitted_model(x, y)

# fig = plt.figure(figsize=(15,5))

# ax = fig.add_subplot(131)
# ax.imshow(image.data, cmap='gray', origin='lower', vmax=0.05)
#            #norm=simple_norm(image.data, stretch='log', log_a=10000))
# ax.set_title('Original image')

# ax = fig.add_subplot(132)
# ax.imshow(image_model, cmap='gray', origin='lower', vmax=0.05)
#           # norm=simple_norm(image.data, stretch='log', log_a=10000))
# ax.set_title('Fitted model')

# ax = fig.add_subplot(133)
# residual = image.data - image_model
# ax.imshow(residual, cmap='gray', origin='lower', vmax=0.05)
#          #  norm=simple_norm(residual, stretch='linear'))
# ax.set_title('Residual')

